<a href="https://colab.research.google.com/github/xinle1030/RecipeTransformer/blob/Jon/FIT3170.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import spacy
from nltk import chunk,pos_tag
from spacy.lang.en import English
parser = English()
nlp = spacy.load("en_core_web_sm")
import json
import re

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **1. Get and clean synonyms and save to synonyms.json file**

In [ ]:
# Get Synonyms for attribute fucntion 
def get_synonyms(word: str):
    synonyms = []
    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            synonyms.append(i.name())
    synonyms.append(word)
    return set(synonyms) 

# Remove unused synonyms from list of synonyms
def clean_synonyms(synonym_list, unused_synonyms):
  for synonym in unused_synonyms:
    if synonym in synonym_list:
      synonym_list.remove(synonym)
  ret_list = []
  for synonym in synonym_list:
    ret_list.append(synonym.lower())
  return ret_list

# Get synonyms for attribute 
cook_time_synonyms = get_synonyms('cook')
serving_synonyms = get_synonyms('Serving')
prep_time_synonyms = get_synonyms('Prep')
ingredient_synonyms = get_synonyms('Ingredients')
instruction_synonyms = get_synonyms('Method')
tips_synonyms = get_synonyms('tips')
nutrient_synonyms = get_synonyms('Nutrition')
cuisine_synonyms = get_synonyms('Cuisine')
category_synonyms = get_synonyms('Categories')

# Remove some synonyms from wordnet 
cook_time_synonyms = clean_synonyms(cook_time_synonyms, ['James_Cook', 'Captain_Cook', 'Captain_James_Cook', 'wangle', 'ready', 'falsify', 'make', 'Cook', 'prepare', 'misrepresent', 'fake', 'manipulate', 'fix', 'fudge'])
serving_synonyms = clean_synonyms(serving_synonyms, ['attend_to', 'suffice', 'dish_out', 'portion', 'helping', 'function', 'wait_on', 'answer', 'swear_out', 'attend', 'do', 'help', 'process', 'serving', 'dish'])
serving_synonyms.append("serves")
prep_time_synonyms = clean_synonyms(prep_time_synonyms, ['homework'])
ingredient_synonyms = clean_synonyms(ingredient_synonyms, ['fixings'])
instruction_synonyms = clean_synonyms(instruction_synonyms, ['method_acting'])
tips_synonyms = clean_synonyms(tips_synonyms, ['tippytoe', 'gratuity', 'lead', 'bakshish', 'steer', 'lean', 'summit', 'crest', 'backsheesh', 'tiptoe', 'baksheesh', 'confidential_information', 'hint', 'angle', 'slant', 'topple', 'tumble', 'top', 'wind', 'bakshis', 'crown', 'fee', 'tap', 'tip_off', 'tilt', 'bung', 'pourboire'])
nutrient_synonyms = clean_synonyms(nutrient_synonyms, ['victuals'])
cuisine_synonyms = clean_synonyms(cuisine_synonyms, [])
category_synonyms = clean_synonyms(category_synonyms, [])

# Combine all synonyms in a single list
list_of_attribute = [serving_synonyms, prep_time_synonyms, ingredient_synonyms, instruction_synonyms,
                     tips_synonyms, nutrient_synonyms, cuisine_synonyms, category_synonyms]


In [ ]:
# Write value stored in list_of_attribute to synonyms.json file

with open('important_files/synonyms.json', 'r') as f:
  json_object = json.load(f)
  for i, key in enumerate(json_object):
    json_object[key] = list(list_of_attribute[i])

with open('synonyms.json', 'w') as f:
    json.dump(json_object, f, indent=2)

# **2a. Read Input Text File (Using file upload)**

In [ ]:
# Read input file and store text from uploaded text file into "text"
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    text = uploaded.get(fn).decode(errors='replace')

Saving recipe_file_26.json to recipe_file_26.json


# **2b. Read Input Text File (Using current saved file)**

In [ ]:
# open text file in read mode
text_file = open('important_files/input_text.txt', 'rb')
text = text_file.read().decode(errors='replace')

# close files
text_file.close()


# **Unzip important_files.zip**

In [ ]:
# Unzip important_files.zip into main directory 
#!unzip important_files.zip


# **3. Text Cleaning**

In [ ]:
# Text Cleaning

# Remove non-ascii characters
def remove_non_ascii(text):
    return re.sub('[^-*.A-Za-z0-9,! ]+', '', text)

# Make the lines into a list  
clean_text_lines = []
flag = True
for line in text.splitlines():
    clean_text_lines.append(line.replace("\n", "").strip())   # remove newline character and space


# **4. Carry Out Division of Categories**

In [ ]:
# Read synonyms and recipe template 
with open('important_files/synonyms.json', 'r') as f:
  syn_json_object = json.load(f)

with open('important_files/recipe_template.json') as f:
  data = json.load(f)

# Append the recipe template to recipe_file.json (output json file)
with open('important_files/recipe_file.json', 'w') as f:
  json.dump(data, f, indent=2)

# Read the updated recipe_file.json file
with open('important_files/recipe_file.json', 'r') as f:
  recipe_json_object = json.load(f)

# Carry out division
result = []

for index, line in enumerate(clean_text_lines):
  for key in syn_json_object:
    # Division for Tips
    if key == "tips" and key in line.lower():
      result.append((key, index))
    # Division for Serving, Ingredient, Instructions, Nutrients
    else:
      for attribute in syn_json_object[key]:
        if (attribute in line.lower()) and (key in recipe_json_object) and (not recipe_json_object[key]):
          result.append((key, index))
          recipe_json_object[key] = True

# **5. Categorize all cleaned text and save into recipe_file.json**

In [ ]:
# Write to recipe json file

def categorize(json_obj, result):
    # Variable to flag whether the title has been stored in recipe_file.json file or not
    title_done = False

    # If there's no serving amount listed in recipe, use first few lines before first category's index as title 
    first_category_index = result[0][1] # Get the index of the first category
    if (first_category_index != 0):
      # Store title value in recipe_file.json file     
      json_obj["title"] = clean_text_lines[0: first_category_index]
      title_done = True

    # General formatting 
    for i, item in enumerate(result):
      key = item[0]
      index = item[1]
      if (key == "serving"):
        serving_arr = re.findall(r'\d+', clean_text_lines[index])
        
        if len(serving_arr) > 0:  # If serving size exists
          serving = serving_arr[0]
          next_item = result[i + 1]
        else:
          serving = 0
          next_item = result[i]
        json_obj[key] = serving

        # If title already stored in recipe_file.json file, do not do title formatting
        if not title_done:     
          # Store title value in recipe_file.json file        
          next_index = next_item[1]
          json_obj["title"] = clean_text_lines[index + 1 : next_index]
      
      # When the last important category is reached, exclude page label and
      # append the remaining categories and their values 
      elif ((i + 1) >= len(result)):  
        last_index = len(clean_text_lines) - 3  # Exclude page label
        json_obj[key] = clean_text_lines[index + 1 : last_index]
      else:
        next_item = result[i + 1]
        next_index = next_item[1]
        json_obj[key] = clean_text_lines[index + 1 : next_index]
    return json_obj

# Clear all non-relevant characters
def clear_text(text):
    return re.sub('[^-.A-Za-z0-9*,! ]+', '', text)

def format_steps(str_list):
  ret = []
  current = []
  for i in range(0, len(str_list)):
    string = str_list[i]
    if bool(re.match('^[\.a-zA-Z0-9,! ]*$', string[0])):
      current.append(string)
    else:
      step = " ".join(current)
      ret.append(clear_text(step.strip()))
      current = [string]
    if i == len(str_list) - 1:
      step = " ".join(current)
      ret.append(clear_text(step.strip()))
  if len(ret) > 0:
    ret.pop(0)
  return ret

# Join strings in the same category
def join_string(json_obj, key):
  # formatting title
  if key in json_obj:
    key_list = json_obj[key]
    if (key == "instruction") or (key == "tips") or (key == "ingredient"):
      json_obj[key] = format_steps(key_list)
    else:
      json_obj[key] = ' '.join(key_list)
  return json_obj

# Append categorized strings to their respective category keys 
# using the template in recipe_template.json
with open('important_files/recipe_template.json') as f:
    data = json.load(f)
    data = categorize(data, result)
    data = join_string(data, "title")
    data = join_string(data, "ingredient")
    data = join_string(data, "instruction")
    data = join_string(data, "tips")

# Save updated json to recipe_file.json
with open('important_files/recipe_file.json', 'w') as f:
    json.dump(data, f, indent=2)
        

# **6. Format nutrients value in recipe_file.json file**

In [ ]:
# Format nutrients element into properly formatted JSON object

with open('important_files/recipe_file.json', 'r') as f:
    i = 0
    result = {}
    json_object = json.load(f)
    nutrient = json_object["nutrient"]
  
   # Loop through all different nutrients
    while i < len(nutrient):
        searched_val = re.search(r"\d", nutrient[i])  # Search for a numeric value in each nutrient

        # If the element contains key and value (split and separate into key and value)
        if searched_val: 
            index_firstnum = searched_val.start()
            if index_firstnum != 0: # both key and value
                key = nutrient[i][0:(index_firstnum-1)].strip() 
                value = nutrient[i][index_firstnum:len(nutrient[i])].strip()
                
                # Fix when bracket is in value instead of key
                if key[len(key)-1] == "(":
                    key = key[:len(key)]
                    right_brack_index = value.find(")")
                    key = key + value[:right_brack_index+1]
                    value = value[right_brack_index+1:len(value)].strip()
                i += 1
                
        else: # only key (make into key)
            # Fix issue where 0mg is read as Omg 
            if "Omg" in nutrient[i]: 
                split = nutrient[i].split(" ", 1)
                key = split[0]
                value = "0mg"
                i += 1
            else:
                if (i >= len(nutrient)) or ((i + 1) >= len(nutrient)):
                  break # Break while loop if a word has been identified wrongly as a nutrient key
                key = nutrient[i]
                value = nutrient[i + 1]
                i += 2
        result[key] = value               
    
    # Replace value of nutrient key with correctly formatted JSON 
    json_object["nutrient"] = result

# Save updated json to recipe_file.json
with open('important_files/recipe_file.json', 'w') as f:
    json.dump(json_object, f, indent=2)


In [ ]:
# Nutrient